# Welcome to PySyft

The goal of this notebook is to provide step by step explanation of the internal workings of PySyft for developers and have working examples of the API to play with.

**Note:** You should be able to run these without any issues. This notebook will be automatically run by CI and flagged if it fails. If your commit breaks this notebook, either fix the issue or add some information here for others.

In [1]:
assert True is True

In [2]:
import sys
import pytest
import syft as sy
from syft.core.node.common.service.auth import AuthorizationException
from syft.util import key_emoji
sy.LOG_FILE = "syft_do.log"
sy.logger.remove()
_ = sy.logger.add(sys.stdout, level="DEBUG")

Bob decides has some data he wants to share with Alice using PySyft.
The first thing Bob needs to do is to create a Node to handle all the PySyft services he will need to safely and securely share his data with Alice.

In [3]:
some_device = sy.Device()

[2021-05-06T13:40:22.306660+0300][DEBUG][logger]][448868] > Creating 🗃️ (MemoryStore) {}
[2021-05-06T13:40:22.307426+0300][DEBUG][logger]][448868] Created MemoryStore.
[2021-05-06T13:40:22.308213+0300][DEBUG][logger]][448868] > Creating 📱 Trusting Kilby (Device)@<UID:🚱🙔>


This is a device, it is a Node and it has a name, id and address.

In [4]:
print(some_device.name, some_device.id, some_device.address)

Trusting Kilby <UID: ab8498b778ec497e8a0644752808e4c3> <Address - Device:<SpecificLocation:..8e4c3>>


The ID is a class called UID which is essentially a uuid. The address is a combination of up to four different locations, identifying the path to resolve the final target of the address.

In [5]:
print(some_device.address.vm, some_device.address.device, some_device.address.domain, some_device.address.network)
print(some_device.address.target_id)

None <SpecificLocation: ab8498b778ec497e8a0644752808e4c3> None None
<SpecificLocation: ab8498b778ec497e8a0644752808e4c3>


UIDs are hard to read and compare so we have a function which converts them to 2 x emoji.
Just like the "name", Emoji uniqueness is not guaranteed but is very useful during debugging.

In [6]:
print(some_device.address.target_id.pprint)
print(some_device.address.pprint)

📌 Trusting Kilby (SpecificLocation)@<UID:🚱🙔>
💠 [📱] Trusting Kilby (Address)


Most things that are "pretty printed" include an emoji, a name and a class, additionally with a UID and "visual UID" emoji.  
📌 some_device.address.target_id is a "Location" which is pointing to this specific location with a name and address.  
💠 [📱] some_device.address is the "Address" of some_device (Think up to 4 x locations) in this case the contents of the List [] show it only has the Location of a device currently.

**note:** Sometimes Emojis look like this: 🙓. The dynamically generated code point does not have an Emoji. A PR to fix this, would be welcome. 🙂

In [7]:
print(some_device.id, some_device.address.target_id.id)
print(some_device.id.emoji(), "==", some_device.address.target_id.id.emoji())
print(some_device.pprint)
assert some_device.id == some_device.address.target_id.id

<UID: ab8498b778ec497e8a0644752808e4c3> <UID: ab8498b778ec497e8a0644752808e4c3>
<UID:🚱🙔> == <UID:🚱🙔>
📱 Trusting Kilby (Device)@<UID:🚱🙔>


Interaction with a Node like a device is always done through a client. Clients can "send" messages and Nodes can "receive" them. Bob needs to get a client for his device. But first it might be a good idea to name the device so that it's easier to follow.

In [8]:
bob_device = sy.Device(name="Bob's iPhone")
assert bob_device.name == "Bob's iPhone"

[2021-05-06T13:40:22.339583+0300][DEBUG][logger]][448868] > Creating 🗃️ (MemoryStore) {}
[2021-05-06T13:40:22.340439+0300][DEBUG][logger]][448868] Created MemoryStore.
[2021-05-06T13:40:22.341291+0300][DEBUG][logger]][448868] > Creating 📱 Bob's iPhone (Device)@<UID:🚁🚃>


In [9]:
bob_device_client = bob_device.get_client()

[2021-05-06T13:40:22.427264+0300][INFO][logger]][448868] Skipping torch.Tensor.xpu not supported in 1.7.1
[2021-05-06T13:40:22.427910+0300][INFO][logger]][448868] Skipping torch.Tensor.tile not supported in 1.7.1
[2021-05-06T13:40:22.428416+0300][INFO][logger]][448868] Skipping torch.Tensor.fmax not supported in 1.7.1
[2021-05-06T13:40:22.428886+0300][INFO][logger]][448868] Skipping torch.Tensor.ldexp_ not supported in 1.7.1
[2021-05-06T13:40:22.429348+0300][INFO][logger]][448868] Skipping torch.Tensor.sinc not supported in 1.7.1
[2021-05-06T13:40:22.429875+0300][INFO][logger]][448868] Skipping torch.kron not supported in 1.7.1
[2021-05-06T13:40:22.430426+0300][INFO][logger]][448868] Skipping torch.Tensor.nan_to_num not supported in 1.7.1
[2021-05-06T13:40:22.431027+0300][INFO][logger]][448868] Skipping torch.msort not supported in 1.7.1
[2021-05-06T13:40:22.431615+0300][INFO][logger]][448868] Skipping torch.row_stack not supported in 1.7.1
[2021-05-06T13:40:22.432147+0300][INFO][logge

When you ask a node for a client you get a "Client" which is named after the device and has the same "UID" and "Address" (4 x Locations) as the device it was created from, and it will have a "Route" that connects it to the "Device"

In [10]:
assert bob_device_client.name == "Bob's iPhone Client"

In [11]:
print(bob_device_client.pprint, bob_device.pprint)
print(bob_device.id.emoji(), "==", bob_device_client.id.emoji())
assert bob_device.id == bob_device_client.device.id
assert bob_device.address == bob_device_client.address

📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> 📱 Bob's iPhone (Device)@<UID:🚁🚃>
<UID:🚁🚃> == <UID:🚁🚃>


📡 [📱] Bob's iPhone Client is a "DeviceClient" and it has the same UID and "Address" as Bob's Device

Now we have something that can send and receive lets take it for a spin.
Since everything is handled with a layer of abstraction the smallest unit of work is a "SyftMessage". Very little can be done without sending a message from a Client to a Node. There are many types of "SyftMessage" which boil down to whether or not they are Sync or Async, and whether or not they expect a response.

Lets make a ReprMessage which simply gets a message and prints it at its destination Node.
SyftMessage's all have an "address" field, without this they would never get delivered. They also generally have a msg_id which can be used to keep track of them.

In [12]:
msg = sy.ReprMessage(address=bob_device_client.address)
print(msg.pprint)
print(bob_device_client.address.pprint)
assert msg.address == bob_device_client.address

[2021-05-06T13:40:22.700621+0300][DEBUG][logger]][448868] > Creating ✉️  (ReprMessage) <UID:🚂🚃>
✉️  (ReprMessage)
💠 [📱] Bob's iPhone Client (Address)


What type of Message is ReprMessage you ask?

In [13]:
print(sy.ReprMessage.mro())

[<class 'syft.core.node.common.service.repr_service.ReprMessage'>, <class 'syft.core.common.message.ImmediateSyftMessageWithoutReply'>, <class 'syft.core.common.message.ImmediateSyftMessage'>, <class 'syft.core.common.message.SyftMessageWithoutReply'>, <class 'syft.core.common.message.SyftMessage'>, <class 'syft.core.common.message.AbstractMessage'>, <class 'syft.core.common.object.ObjectWithID'>, <class 'syft.core.common.serde.serializable.Serializable'>, <class 'typing.Generic'>, <class 'object'>]


It's an "Immediate" "WithoutReply" okay so Sync and no response.

Now lets send it, remember we need a Client not a Node for sending.

In [14]:
with pytest.raises(AuthorizationException):
    bob_device_client.send_immediate_msg_without_reply(
        msg=sy.ReprMessage(address=bob_device_client.address)
    )

[2021-05-06T13:40:22.714418+0300][DEBUG][logger]][448868] > Creating ✉️  (ReprMessage) <UID:🚃🚀>
[2021-05-06T13:40:22.715596+0300][DEBUG][logger]][448868] > 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> Signing ✉️  (ReprMessage) with 🜾
[2021-05-06T13:40:22.716444+0300][DEBUG][logger]][448868] > Signing with 🜾
[2021-05-06T13:40:22.718036+0300][DEBUG][logger]][448868] > Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚃🚀>
[2021-05-06T13:40:22.718857+0300][DEBUG][logger]][448868] > Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
[2021-05-06T13:40:22.719644+0300][DEBUG][logger]][448868] > Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
[2021-05-06T13:40:22.720942+0300][DEBUG][logger]][448868] > Creating ✉️  (ReprMessage) <UID:🚃🚀>
[2021-05-06T13:40:22.721960+0300][DEBUG][logger]][448868] > Received without Reply ✉️  (ReprMessage) <UID: 150b3b8480e441

Oh oh! Why did Auth fail? We'll we can see from the debug that the 🔑 (VerifyKey) of the sender was matched to the 🗝 (Root VerifyKey) of the destination and they don't match. This client does not have sufficient permission to send a ReprMessage.

First lets take a look at the keys involved.

In [15]:
print(bob_device_client.keys)
print(bob_device.keys)

assert bob_device_client.verify_key != bob_device.root_verify_key

🔑 🜾
🔑 🛁🗝 🛁


Not to worry we have a solution, lets get a client which does have this permission.

In [16]:
bob_device_client = bob_device.get_root_client()

[2021-05-06T13:40:22.824306+0300][INFO][logger]][448868] Skipping torch.Tensor.xpu not supported in 1.7.1
[2021-05-06T13:40:22.824947+0300][INFO][logger]][448868] Skipping torch.Tensor.tile not supported in 1.7.1
[2021-05-06T13:40:22.825431+0300][INFO][logger]][448868] Skipping torch.Tensor.fmax not supported in 1.7.1
[2021-05-06T13:40:22.825860+0300][INFO][logger]][448868] Skipping torch.Tensor.ldexp_ not supported in 1.7.1
[2021-05-06T13:40:22.826299+0300][INFO][logger]][448868] Skipping torch.Tensor.sinc not supported in 1.7.1
[2021-05-06T13:40:22.826817+0300][INFO][logger]][448868] Skipping torch.kron not supported in 1.7.1
[2021-05-06T13:40:22.827339+0300][INFO][logger]][448868] Skipping torch.Tensor.nan_to_num not supported in 1.7.1
[2021-05-06T13:40:22.827828+0300][INFO][logger]][448868] Skipping torch.msort not supported in 1.7.1
[2021-05-06T13:40:22.828404+0300][INFO][logger]][448868] Skipping torch.row_stack not supported in 1.7.1
[2021-05-06T13:40:22.829910+0300][INFO][logge

Lets take a look again.

In [17]:
print(bob_device_client.keys)
print(bob_device.keys)

assert bob_device_client.verify_key == bob_device.root_verify_key

🔑 🛲
🔑 🛁🗝 🛲


Lets try sending the message again.

In [18]:
bob_device_client.send_immediate_msg_without_reply(
    msg=sy.ReprMessage(address=bob_device_client.address)
)

[2021-05-06T13:40:22.973935+0300][DEBUG][logger]][448868] > Creating ✉️  (ReprMessage) <UID:🚴🚆>
[2021-05-06T13:40:22.974595+0300][DEBUG][logger]][448868] > 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> Signing ✉️  (ReprMessage) with 🛲
[2021-05-06T13:40:22.975141+0300][DEBUG][logger]][448868] > Signing with 🛲
[2021-05-06T13:40:22.976270+0300][DEBUG][logger]][448868] > Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚴🚆>
[2021-05-06T13:40:22.976830+0300][DEBUG][logger]][448868] > Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
[2021-05-06T13:40:22.977364+0300][DEBUG][logger]][448868] > Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
[2021-05-06T13:40:22.978361+0300][DEBUG][logger]][448868] > Creating ✉️  (ReprMessage) <UID:🚴🚆>
[2021-05-06T13:40:22.978921+0300][DEBUG][logger]][448868] > Received without Reply ✉️  (ReprMessage) <UID: 4d944d30467841

Woohoo! 🎉

Okay so there's a lot going on but lets step through it.  
The ReprMessage is created by the Client, and then signed with the Client's SigningKey.
The SigningKey and VerifyKey are a pair and the VerifyKey is public and derived from the SigningKey.
When we call get_root_client() we update the Node with the newly generated key on the Client so that the Client will now have permission to execute root actions.

Behind every message type is a service which executes the message on a Node.
To run ReprMessage on our Device Node, we can see that during startup it adds a service to handle these kinds of messages:
```python
# common/node.py
self.immediate_services_without_reply.append(ReprService)
````

Not all actions / services require "root". To enable this a decorator is added like so:
```python
# repr_service.py
class ReprService(ImmediateNodeServiceWithoutReply):
    @staticmethod
    @service_auth(root_only=True)
```

Okay so Bob has root access to his own device, but he wants to share some data and compute resources of this device to someone else. So to do that he needs to create a "Sub Node" which will be a "VirtualMachine". Think of this as a partition or slice of his device which can be allocated memory, storage and compute.

In [19]:
bob_vm = sy.VirtualMachine(name="Bob's VM")

[2021-05-06T13:40:22.988158+0300][DEBUG][logger]][448868] > Creating 🗃️ (MemoryStore) {}
[2021-05-06T13:40:22.988939+0300][DEBUG][logger]][448868] Created MemoryStore.
[2021-05-06T13:40:22.990067+0300][DEBUG][logger]][448868] > Creating 🍰 Bob's VM (VirtualMachine)@<UID:🙏🚃>


Since VirtualMachine is a Node (Server) it will need a Client to issue commands to it.
Lets make one.  
**note:** Why do we need a root client? The registration process is two-way and the Registered will need to update its address in response to a successful registration.

In [20]:
bob_vm_client = bob_vm.get_root_client()

[2021-05-06T13:40:23.093578+0300][INFO][logger]][448868] Skipping torch.Tensor.xpu not supported in 1.7.1
[2021-05-06T13:40:23.094247+0300][INFO][logger]][448868] Skipping torch.Tensor.tile not supported in 1.7.1
[2021-05-06T13:40:23.094732+0300][INFO][logger]][448868] Skipping torch.Tensor.fmax not supported in 1.7.1
[2021-05-06T13:40:23.095491+0300][INFO][logger]][448868] Skipping torch.Tensor.ldexp_ not supported in 1.7.1
[2021-05-06T13:40:23.096703+0300][INFO][logger]][448868] Skipping torch.Tensor.sinc not supported in 1.7.1
[2021-05-06T13:40:23.097333+0300][INFO][logger]][448868] Skipping torch.kron not supported in 1.7.1
[2021-05-06T13:40:23.098024+0300][INFO][logger]][448868] Skipping torch.Tensor.nan_to_num not supported in 1.7.1
[2021-05-06T13:40:23.098675+0300][INFO][logger]][448868] Skipping torch.msort not supported in 1.7.1
[2021-05-06T13:40:23.099363+0300][INFO][logger]][448868] Skipping torch.row_stack not supported in 1.7.1
[2021-05-06T13:40:23.099975+0300][INFO][logge

Okay so now Bob has two Nodes and their respective clients, but they know nothing of each other.
They both have addresses that only point to themselves.

In [21]:
print(bob_device_client.address.pprint)
print(bob_vm_client.address.pprint)

💠 [📱] Bob's iPhone Client (Address)
💠 [🍰] Bob's VM Client (Address)


Lets register Bob's vm with its device since the Device is higher up in the level of scope.

In [22]:
bob_device_client.register(client=bob_vm_client)

[2021-05-06T13:40:23.330728+0300][DEBUG][logger]][448868] > Registering 📡 [🍰] Bob's VM Client (VirtualMachineClient)@<UID:🙏🚃> with 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃>
[2021-05-06T13:40:23.331775+0300][DEBUG][logger]][448868] > Creating ✉️  (RegisterChildNodeMessage) <UID:🚵🚹>
[2021-05-06T13:40:23.332411+0300][DEBUG][logger]][448868] > 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> Signing ✉️  (RegisterChildNodeMessage) with 🛲
[2021-05-06T13:40:23.332941+0300][DEBUG][logger]][448868] > Signing with 🛲
[2021-05-06T13:40:23.333337+0300][DEBUG][logger]][448868] > ✉️  -> Proto 🔢
[2021-05-06T13:40:23.334530+0300][DEBUG][logger]][448868] > Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🚵🚹>
[2021-05-06T13:40:23.335251+0300][DEBUG][logger]][448868] > Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚁🚃> ➡️  💠 [📱] Bob's iPhone Client (Address)
[2021-05-06T13:40:23.335780+0300][DEBUG][logger]][448868] > Routing ✉

Whoa.. Okay lots happening. As you can see there are two messages and two Authentications. The first one is the `RegisterChildNodeMessage` which is dispatched to the address of the Device. Once it is received it stores the address of the registering Node and then dispatches a new `HeritageUpdateMessage` back to the sender of the first message.

**note:** This is not a reply message, this is a completely independent message that happens to be sent back to the sender's address.
```python
issubclass(RegisterChildNodeMessage, SignedImmediateSyftMessageWithoutReply)
issubclass(HeritageUpdateService, SignedImmediateSyftMessageWithoutReply)
```

You will also notice that the Messages turned into Protobufs and then Signed.  
Lets see how this works for `ReprMessage`.

1) ✉️  -> Proto 🔢

Every message that is sent requires and the following method signatures:
```python
def _object2proto(self) -> ReprMessage_PB:
def _proto2object(proto: ReprMessage_PB) -> "ReprMessage":
def get_protobuf_schema() -> GeneratedProtocolMessageType:
```

The get_protobuf_schema method will tell the caller what Protobuf class to use, and then the _object2proto method will be called to turn normal python into a protobuf message.

```python
# repr_service.py
def _object2proto(self) -> ReprMessage_PB:
    return ReprMessage_PB(
        msg_id=serialize(self.id), address=serialize(self.address),
    )
```

Any type which isn't a normal Protobuf primitive must be converted to a proto or serialized before being stored.
**note:** self.id and self.address also need to be serialized so this will call their `_object2proto` methods.  

At this point we are using code auto generated by `protoc` as per the build script:
```bash
$ ./scripts/build_proto.sh
```

Here is the .proto definition for `ReprMessage`

```c++
// repr_service.proto
syntax = "proto3";

package syft.core.node.common.service;

import "proto/core/common/common_object.proto";
import "proto/core/io/address.proto";

message ReprMessage {
    syft.core.common.UID msg_id = 1;
    syft.core.io.Address address = 2;
}
```


Once the message needs to be Deserialized the `_proto2object` method will be called.

```python
# repr_service.py
return ReprMessage(
    msg_id=_deserialize(blob=proto.msg_id),
    address=_deserialize(blob=proto.address),
)
```

Two things to pay attention to:
1) `RegisterChildNodeMessage` has caused Bob's Device to store an entry representing Bob's VM Address  
2) `HeritageUpdateService` has caused Bob's VM to update its address to now include the `SpecificLocation` of Bob's Device.

**note:** The Address for Bob's VM Client inside the Store does not include the "Device" part of the "Address" (4 x Locations) since it isn't updated until after the HeritageUpdateService message is sent.

In [23]:
print(bob_device.store.pprint)
assert bob_vm_client.address.target_id.id in bob_device.store
print(bob_vm_client.address.pprint, bob_vm_client.address.target_id.id.emoji())

🗃️ (MemoryStore) {
  > <UID: 685998c5ab374649a068d1da43e58221> <UID:🙏🚃> => 🗂️ (StorableObject) (💠 [🍰] Bob's VM Client (Address))
}
💠 [🍰📱] Bob's VM Client (Address) <UID:🙏🚃>


What about `SignedMessage`? If you read `message.py` you will see that all messages inherit from `SignedMessage` and as such contain the following fields.

```c++
message SignedMessage {
  syft.core.common.UID msg_id = 1;
  string obj_type = 2;
  bytes signature = 3;
  bytes verify_key = 4;
  bytes message = 5;
}
```

The actual message is serialized and stored in the `message` field and a hash of its bytes is calculated using the Client's `SigningKey`.

The contents of `message` are not encrypted and can be read at any time by simply deserializing them.

In [24]:
def get_signed_message_bytes() -> bytes:
    # return a signed message fixture containing the uid from get_uid
    blob = (
        b"\xfa\x02\x8b\x02\n\x12\n\x10\xbaU\x10\x0c`\xbcB\x06\x89\xa3\xba\x7f\n\xb4\xb8\x93"
        b"\x126syft.core.node.common.service.repr_service.ReprMessage\x1a@h;^#a\xbb\x8f\x9f"
        b"\xad\xde\xf0i\xb4\xfa\x10\xab\t\xe1qt\xb7\xbe\xd9\xba\x1di\x7f\xc5+\xbf(\xa6\x86"
        b'\xe96\xf5\xc9\x86\xad<\xb1\x18p\xe2T\xaf\xbd\xb6h\xdd\xafL\xd9\x0cg\xe5\xe3\xe5"h}'
        b'\xf5\x1e\x07" \xbd\\\xfe\x89\t\xa20\rp\xdf\xcb\xf6\x15\xfe\xa1U\r\x9a\xf1.]'
        b"\xfb4\x1cAH\xe9\x7f.\x17\xb6-*Y\xe2\x01V\n\x12\n\x10\xbaU\x10\x0c`\xbcB\x06\x89\xa3"
        b"\xba\x7f\n\xb4\xb8\x93\x12@\n\x12Zealous Montalcini8\x01B(\n\x12\n\x10P\xf2sP\x81"
        b"\xa5A\xea\xb5F\xe3J\x0e\xa0\x98\xa2\x12\x12Zealous Montalcini"
    )
    return blob

In [25]:
sig_msg = sy.deserialize(blob=get_signed_message_bytes(), from_bytes=True)

DecodeError: Error parsing message

We can get the nested message with a property called `message`

In [ ]:
repr_msg = sig_msg.message
print(repr_msg.pprint, sig_msg.pprint)
print(repr_msg.address.pprint, sig_msg.address.pprint)
print(repr_msg.address.target_id.id.emoji(), sig_msg.address.target_id.id.emoji())
assert sig_msg.id == repr_msg.id
assert sig_msg.address == repr_msg.address

Notice the UID's of `ReprMessage` and `SignedImmediateSyftMessageWithoutReply` are the same. So are the delivery addresses.

But the original bytes are still available and serialization / deserialization or serde (ser/de) is bi-directional and reversible

In [ ]:
assert sy.serialize(repr_msg, to_bytes=True) == sig_msg.serialized_message
print(repr_msg.pprint, " ⬅️ ", sig_msg.pprint)

In [ ]:
from nacl.signing import SigningKey, VerifyKey
def get_signing_key() -> SigningKey:
    # return a the signing key used to sign the get_signed_message_bytes fixture
    key = "e89ff2e651b42393b6ecb5956419088781309d953d72bd73a0968525a3a6a951"
    return SigningKey(bytes.fromhex(key))

Lets try re-signing it with the same key it was signed with.

In [ ]:
sig_msg_comp = repr_msg.sign(signing_key=get_signing_key())

In [ ]:
signing_key = get_signing_key()
verify_key = signing_key.verify_key

print(f"SigningKey: {key_emoji(key=signing_key)}")
print(f"VerifyKey: {key_emoji(key=verify_key)}")
print(type(signing_key), type(verify_key))
print(f"🔑 {key_emoji(key=sig_msg.verify_key)} == {key_emoji(key=verify_key)} 🔑")
print(bytes(verify_key))

assert sig_msg_comp == sig_msg
assert sig_msg.verify_key == verify_key
assert VerifyKey(bytes(verify_key)) == verify_key

The message is signed with the `SigningKey`, a consistent `VerifyKey` is derived from the `SigningKey`. Both keys can be transformed to bytes and back easily.

Okay now Bob wants to protect his Device(s) and its / their VM(s). To do that he needs to add them to a higher level Node called a `Domain`.

In [ ]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_root_client()

Okay lets follow the same procedure and link up these nodes.

In [ ]:
print(bob_domain.address.pprint)

In [ ]:
bob_domain_client.register(client=bob_device_client)

That's interesting, we see there's two `HeritageUpdateMessage` that get sent. The address update is "Flowing" upward to the leaf VM nodes.

In [ ]:
print(bob_vm.address.pprint)
print(bob_device.address.pprint)
print(bob_domain.address.pprint)
assert bob_domain_client.id == bob_device.address.domain.id
assert bob_device.id == bob_vm.address.device.id

Now that the Nodes are aware of each other, we can send a message to any child node by dispatching a message on a parent Client and addressing the Child node.
**note:** We are changing the bob_vm root_verify_key because ReprMessage is a root message. We should change this example.

Note, the repr service has the service_auth decorator.  
`@service_auth(root_only=True)`

```python
class ReprService(ImmediateNodeServiceWithoutReply):
    @staticmethod
    @service_auth(root_only=True)
    def process(node: AbstractNode, msg: ReprMessage, verify_key: VerifyKey) -> None:
        print(node.__repr__())

    @staticmethod
    def message_handler_types() -> List[Type[ReprMessage]]:
        return [ReprMessage]

```

For the purpose of the demonstration we will override the destination nodes root_verify_key with one of our new domain clients and the secure ReprMessage will be executed. Normally the remote node is not running in the same python REPL as the client.

In [ ]:
# Just to bypass the auth we will set the root verify key on destination so that it will accept this message
bob_vm.root_verify_key = bob_domain_client.verify_key  # inject 📡🔑 as 📍🗝
bob_domain_client.send_immediate_msg_without_reply(
    msg=sy.ReprMessage(address=bob_vm.address)
)